In [1]:
%matplotlib inline
import os, sys, inspect, warnings
curr_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) # script directory
main_dir = '/'.join(curr_dir.split('/')[:-1])
sub_dir = os.path.join(main_dir, 'RGCPD/')
# core_pp = os.path.join(main_dir, 'RGCPD/core')
if main_dir not in sys.path:
    sys.path.append(main_dir)
    sys.path.append(sub_dir)
import numpy as np 
from collections import Counter
# import statsmodels.api as sm 
import pandas as pd 
import math
import matplotlib.pyplot as plt 
plt.rcParams['figure.figsize'] = (20.0, 10.0)
import itertools as it
import pywt as wv
from scipy.fftpack import fft
from copy import deepcopy
# from statsmodels.tsa.arima_process import  arma_generate_sample, ArmaProcess
from pprint import pprint as pp 
from pandas.plotting import register_matplotlib_converters
from RGCPD import RGCPD
from RGCPD import BivariateMI
import core_pp
import plot_signal_decomp
import plot_coeffs
from visualize_cwt import *
register_matplotlib_converters()
np.random.seed(12345)
plt.style.use('seaborn')

ModuleNotFoundError: No module named 'RGCPD'

In [ ]:
path_data = os.path.join(main_dir, 'data')
# temporal_data  = os.path.join(main_dir, 'data/Ftemporal')
current_analysis_path = os.path.join(main_dir, 'Jier_analysis')
target= 3
target_path = os.path.join(path_data, 'tf5_nc5_dendo_80d77.nc')
precursor_path = os.path.join(path_data,'z500hpa_1979-2018_1_12_daily_2.5deg.nc')
list_of_name_path = [(target, target_path), 
                    ('wind', precursor_path )]
list_for_MI = [BivariateMI(name='wind', func=BivariateMI.corr_map, 
                          kwrgs_func={'alpha':.0001, 'FDR_control':True}, 
                          distance_eps=700, min_area_in_degrees2=5)]
rg = RGCPD(list_of_name_path=list_of_name_path,
           list_for_MI=list_for_MI,
           start_end_TVdate=('06-01', '08-31'),
           path_outmain=os.path.join(main_dir,'data'))

In [ ]:
rg.pp_precursors(detrend=True, anomaly=True, selbox=None)
rg.pp_TV()
rg.traintest(method='no_train_test_split')
rg.calc_corr_maps()
rg.cluster_list_MI()
rg.get_ts_prec(precur_aggr=1)

In [ ]:
rg.dates_TV

In [ ]:
rg.PCMCI_df_data()
rg.PCMCI_get_links()
rg.df_MCIc
rg.PCMCI_plot_graph()

In [ ]:
rg_data  = rg.df_data[['3ts', '0..1..sm', '0..2..sm']]
rg_data = rg_data.rename(columns={'0..1..sm':'prec1', '0..2..sm':'prec2'})
rg_index = rg_data.index.levels[1]
prec1 = rg_data['prec1'].values
prec2 = rg_data['prec2'].values
target = rg_data['3ts'].values
wave  = wv.Wavelet('db4')
mode=wv.Modes.periodic

In [ ]:
plt.figure(figsize=(19,8), dpi=120)
plt.plot(rg_index, prec1)
plt.show()

In [ ]:
plt.figure(figsize=(19,8), dpi=120)
plt.plot(rg_index, prec2)
plt.show()

In [ ]:
fig, ax = plt.subplots(wv.dwt_max_level(len(prec1), wave.dec_len), 2, figsize=(19, 8))
fig.suptitle('Using Discrete Wavelet transform', fontsize=14)
ap = rg_data['prec1'].values
for i in range(wv.dwt_max_level(len(prec1), wave.dec_len)):
   ap, det =  wv.dwt(ap, 'db4')
   ax[i, 0].plot(ap, 'r')
   ax[i, 1].plot(det, 'g')
   ax[i, 0].set_ylabel('Level {}'.format(i + 1), fontsize=9, rotation=90)
   if i == 0:
        ax[i, 0].set_title('Approximation coeffs', fontsize=14)
        ax[i, 1].set_title('Details coeffs', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(wv.dwt_max_level(len(prec2), wave.dec_len), 2, figsize=(19, 8))
fig.suptitle('Using Discrete Wavelet transform', fontsize=14)
ap = rg_data['prec2'].values
for i in range(wv.dwt_max_level(len(prec2), wave.dec_len)):
   ap, det =  wv.dwt(ap, 'db4')
   ax[i, 0].plot(ap, 'r')
   ax[i, 1].plot(det, 'g')
   ax[i, 0].set_ylabel('Level {}'.format(i + 1), fontsize=9, rotation=90)
   if i == 0:
        ax[i, 0].set_title('Approximation coeffs', fontsize=14)
        ax[i, 1].set_title('Details coeffs', fontsize=14)
plt.tight_layout()
plt.show()